# Automatic Differentiation: Make the computer do it

In one implementation of *forward-mode* automatic differentiation (autodiff), we use "dual numbers" to carry forward partial derivatives along with our calculation. A dual number is similar to a complex number in that it has a real valued component, and an extra component. In the case of dual numbers the extra component is an "infinitesimal" component $\epsilon$. Whereas in complex numbers, $i$ is defined by $i^2 = -1$, for dual numbers, $\epsilon$ is defined by $\epsilon^2 = 0$.

In [18]:
using BenchmarkTools

In [1]:
# create a dual number type
immutable Dual
    value::Float64
    eps::Float64
end

We'll attempt to differentiate the function

$$f(x) = x^2 + x \sin(x)$$

which seems moderately interesting.

In [2]:
# define a test function:
f(x) = x^2 + x * sin(x)

f (generic function with 1 method)

In [3]:
f(1.)

1.8414709848078965

So we'll need to know how to multiply, add and take the sine of our `Dual` type.

In [4]:
import Base: +, *, sin

In [5]:
*(x::Dual, y::Dual) = Dual(x.value * y.value, x.value * y.eps + y.value * x.eps)

* (generic function with 152 methods)

In [6]:
+(x::Dual, y::Dual) = Dual(x.value + y.value, x.eps + y.eps)

+ (generic function with 164 methods)

In [7]:
sin(x::Dual) = Dual(sin(x.value), cos(x.value) * x.eps)

sin (generic function with 11 methods)

Sweet! Now we'll try to run it!

In [ ]:
println(f(3.))

In [20]:
@benchmark f(Dual(3., 1.))

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     976
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  96.00 bytes
  allocs estimate:  3
  minimum time:     81.00 ns (0.00% GC)
  median time:      82.00 ns (0.00% GC)
  mean time:        89.80 ns (6.19% GC)
  maximum time:     2.58 μs (95.16% GC)

Check it:

In [9]:
fprime(x) = 2 * x + sin(x) + x * cos(x)

fprime (generic function with 1 method)

In [19]:
@benchmark fprime(3.)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     995
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  0.00 bytes
  allocs estimate:  0
  minimum time:     33.00 ns (0.00% GC)
  median time:      33.00 ns (0.00% GC)
  mean time:        33.22 ns (0.00% GC)
  maximum time:     102.00 ns (0.00% GC)

In [11]:
@code_native f(Dual(3., 1.))

	.text
Filename: In[2]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rbx
	subq	$80, %rsp
	movq	%rsi, %rbx
	movq	%rdi, %r14
	movq	%fs:0, %r12
	addq	$-2672, %r12            # imm = 0xFFFFFFFFFFFFF590
	leaq	-56(%rbp), %r15
	vxorps	%xmm0, %xmm0, %xmm0
	vmovups	%xmm0, -56(%rbp)
	movq	$0, -40(%rbp)
	movq	$8, -80(%rbp)
	movq	(%r12), %rax
	movq	%rax, -72(%rbp)
	leaq	-80(%rbp), %rax
	movq	%rax, (%r12)
	movq	$0, -64(%rbp)
Source line: 2
	movabsq	$power_by_squaring, %rax
	movl	$2, %esi
	movq	%rbx, %rdi
	callq	*%rax
	movq	%rax, -48(%rbp)
Source line: 1
	movabsq	$sin, %rax
	vmovsd	(%rbx), %xmm0           # xmm0 = mem[0],zero
Source line: 2
	vmovsd	%xmm0, -88(%rbp)
	vmovsd	8(%rbx), %xmm1          # xmm1 = mem[0],zero
Source line: 1
	vmovsd	%xmm1, -104(%rbp)
	callq	*%rax
	vmovsd	%xmm0, -112(%rbp)
	movabsq	$cos, %rax
	vmovsd	-88(%rbp), %xmm0        # xmm0 = mem[0],zero
	callq	*%rax
	vmovsd	-104(%rbp), %xmm3       # xmm3 = mem[0],zero
Source line: 2
	vmulsd	%xmm3, %xmm0, %xmm

In [14]:
@code_native fprime(3.)

	.text
Filename: In[9]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	subq	$16, %rsp
	vmovsd	%xmm0, -8(%rbp)
	movabsq	$sin, %rax
	callq	*%rax
	vmovsd	%xmm0, -16(%rbp)
	movabsq	$cos, %rax
	vmovsd	-8(%rbp), %xmm0         # xmm0 = mem[0],zero
	callq	*%rax
	vmovsd	-8(%rbp), %xmm2         # xmm2 = mem[0],zero
	vaddsd	%xmm2, %xmm2, %xmm1
	vaddsd	-16(%rbp), %xmm1, %xmm1
	vmulsd	%xmm2, %xmm0, %xmm0
	vaddsd	%xmm0, %xmm1, %xmm0
	addq	$16, %rsp
	popq	%rbp
	retq
	nopl	(%rax,%rax)
